In [1]:
using BenchmarkTools, Compat, DataFrames, Distributions, ForwardDiff

In [2]:
# Basic trust region with truncated conjugate gradient.

df = readtable("data/model_australia.txt", separator = ' ', header = false)

names!(df, [Symbol("x$i") for i in 1:4])

head(df)

,x1,x2,x3,x4
1,1,0,0,0
2,0,1,0,0
3,0,0,1,0
4,35,0,0,0
5,69,34,35,0
6,70,71,70,30


In [3]:
mixed_logit = DataFrame(P = 1.0:210.0)

names!(mixed_logit, [Symbol("Score")])

storage = DataFrame(P = 1.0:5.0)

names!(storage, [Symbol("Score")])

rand_contdist(rng::AbstractRNG, Dist::Distribution) = quantile(Dist, rand(rng))

rand_contdist (generic function with 1 method)

In [4]:
function individual(β::Vector, i::Int64)
    data = convert(Array, df[i*7-6:i*7-1, :])
    choices = convert(Array, df[i*7:i*7, :])
    alternatives = find(choices .== 0)
    choice = find(choices .== 1)[1]
    
    function utility(β::Vector, i::Int64)
        return dot(vec(data[:, i]), β)
    end
    
    function beta(γ::Vector, θ::Vector, β::Float64 = 0.0)
        for k = 1:length(γ)
            β += θ[k*1]+θ[k*2]*γ[k]
        end
        return β
    end
    
    function probability(β::Vector, t::Float64 = 0.0)
        
        # Je simule pour le β aléatoire. À modifier.
        γ = [rand_contdist(MersenneTwister(100), Uniform())]
        
        #println("γ:    ", γ)
        
        # Je convertit à partir de γ et θ. À modifier.
        θ = [0.0, 1.0]
        k = beta(γ, θ)
        
        temp = β[1:4]
        push!(temp, k)
        push!(temp, β[length(β)])
        
        #println("β:    ", β)
        #println("temp: ", temp)
        
        c = utility(temp, choice)
        for alternative in alternatives
            t += exp(utility(temp, alternative)-c)
        end
        return 1/(1+t)
    end
    
    return probability
end

individual (generic function with 1 method)

In [5]:
function f(β::Vector, model::Float64 = 0.0, n::Int64 = 210)
    i = 1
    while i <= n
        probability = individual(β, i)
        model += log(probability(β))
        i += 1
    end
    return -model/n
end

f (generic function with 3 methods)

In [6]:
immutable BasicTrustRegion{T<:Real}
    η1::T
    η2::T
    γ1::T
    γ2::T
end

function BTRDefaults()
    return BasicTrustRegion(0.01, 0.9, 0.5, 0.5)
end

type BTRState
    iter::Int
    β::Vector
    βcand::Vector
    g::Vector
    step::Vector
    Δ::Float64
    ρ::Float64

    function BTRState()
        return new()
    end
end

In [7]:
function acceptCandidate!(state::BTRState, b::BasicTrustRegion)
    if state.ρ >= b.η1
        return true
    else
        return false
    end
end

function updateRadius!(state::BTRState, b::BasicTrustRegion)
    if state.ρ >= b.η2
        stepnorm = norm(state.step)
        state.Δ = min(1e20, max(4*stepnorm, state.Δ))
    elseif state.ρ >= b.η1
        state.Δ *= b.γ2
    else
        state.Δ *= b.γ1
    end
end

updateRadius! (generic function with 1 method)

In [8]:
function TruncatedCG(g::Vector, H::Matrix, Δ::Float64)
    n = length(g)
    s = zeros(n)
    normg0 = norm(g)
    v = g
    d = -v
    gv = dot(g, v)
    norm2d = gv
    norm2s = 0
    sMd = 0
    k = 0
    Δ = Δ*Δ
    while stopCG(norm(g), normg0, k, n) == false
        Hd = H*d
        κ = dot(d, Hd)
        if κ <= 0
            σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ-dot(s, s))))/norm2d
            s += σ*d
            break
        end
        α = gv/κ
        norm2s += α*(2*sMd+α*norm2d)
        if norm2s >= Δ
            σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ-dot(s, s))))/norm2d
            s += σ*d
            break
        end
        s += α*d
        g += α*Hd
        v = g
        newgv = dot(g, v)
        β = newgv/gv
        gv = newgv
        d = -v+β*d
        sMd = β*(sMd+α*norm2d)
        norm2d = gv+β*β*norm2d
        k += 1
    end
    return s
end

TruncatedCG (generic function with 1 method)

In [9]:
function stopCG(normg::Float64, normg0::Float64, k::Int, kmax::Int)
    χ::Float64 = 0.1
    θ::Float64 = 0.5
    if (k == kmax) || (normg <= normg0*min(χ, normg0^θ))
        return true
    else
        return false
    end
end

stopCG (generic function with 1 method)

In [10]:
function btr(f::Function, g!::Function, H!::Function, Step::Function, β0::Vector,
        state::BTRState = BTRState(), ApproxH::Bool = false)
    b = BTRDefaults()
    state.iter = 0
    state.β = β0
    n = length(β0)
    tol = 1e-6*1e-6
    state.g = zeros(n)
    H = eye(n, n)
    fβ = f(β0)
    g!(β0, state.g)
    state.Δ = 0.1*norm(state.g)
    if ApproxH
        y = zeros(n)
        gcand = zeros(n)
    else
        H!(β0, H)
    end
    nmax = 100
    
    function model(s::Vector, g::Vector, H::Matrix)
        return dot(s, g)+0.5*dot(s, H*s)
    end
    
    while dot(state.g, state.g) > tol && state.iter < nmax
        state.step = Step(state.g, H, state.Δ)
        state.βcand = state.β+state.step
        fcand = f(state.βcand)
        state.ρ = (fcand-fβ)/(model(state.step, state.g, H))
        if ApproxH
            g!(state.βcand, gcand)
            y = gcand-state.g;
            H = H!(H, y, state.step)
        end
        if acceptCandidate!(state, b)
            state.β = copy(state.βcand)
            if ApproxH == false
                g!(state.β, state.g)
                H!(state.β, H)
            else
                state.g = copy(gcand)
            end
            fβ = fcand
        end
        updateRadius!(state, b)
        state.iter += 1
    end
    return state.β, state.iter
end

btr (generic function with 3 methods)

In [11]:
function g(β::Vector, n::Int64 = 210)
    t = zeros(length(β))
    for i = 1:n
        probability = individual(β, i)
        t += (1/probability(β))*ForwardDiff.gradient(probability, β)
    end
    return -t/n
end

function g!(β::Vector, storage::Vector)
    s = g(β)
    storage[1:length(s)] = s[1:length(s)]
end

g! (generic function with 1 method)

In [12]:
function H(β::Vector)
    return ForwardDiff.hessian(f, β)
end

function H!(β::Vector, storage::Matrix)
    s = H(β)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

H! (generic function with 1 method)

In [13]:
function BHHH(β::Vector, n::Int64 = 210)
    t = zeros(length(β), length(β))
    for i = 1:n
        probability = individual(β, i)
        g = ForwardDiff.gradient(probability, β)
        t += g*(g')
    end
    return -t/n
end

function BHHH!(β::Vector, storage::Matrix)
    s = BHHH(β)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

BHHH! (generic function with 1 method)

In [14]:
function BFGS(B::Matrix, y::Vector, s::Vector)
    Bs = B*s
    return B-(Bs*Bs')/dot(s, Bs)+(y*y')/dot(s, y)
end

function BFGS!(B::Matrix, y::Vector, s::Vector)
    n, m = size(B)
    B[1:n, 1:m] = BFGS(B, y, s)
end

BFGS! (generic function with 1 method)

In [15]:
function SR1(B::Matrix, y::Vector, s::Vector)
    Bs = B*s
    return B+((y-Bs)*(y-Bs)')/((y-Bs)'*s)
end

function SR1!(B::Matrix, y::Vector, s::Vector)
    n, m = size(B)
    B[1:n, 1:m] = SR1(B, y, s)
end

SR1! (generic function with 1 method)

In [16]:
@benchmark btr(f, g!, H!, TruncatedCG, [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], BTRState())
btr(f, g!, H!, TruncatedCG, [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], BTRState())

([-18.9371, -7.58557, -11.2609, 0.0541184, 0.0, 1.0, -0.0579395], 14)

In [17]:
@benchmark btr(f, g!, BHHH!, TruncatedCG, [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], BTRState())
btr(f, g!, BHHH!, TruncatedCG, [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], BTRState())

([-0.915939, 0.79032, -1.18917, -0.252563, 0.0, 1.0, -0.180764], 100)

In [18]:
@benchmark btr(f, g!, BFGS!, TruncatedCG, [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], BTRState(), true)
btr(f, g!, BFGS!, TruncatedCG, [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], BTRState(), true)

([-18.9371, -7.58557, -11.2609, 0.0541184, 0.0, 1.0, -0.0579395], 37)

In [19]:
@benchmark btr(f, g!, SR1!, TruncatedCG, [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], BTRState(), true)
btr(f, g!, SR1!, TruncatedCG, [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], BTRState(), true)

([-18.9371, -7.58557, -11.2609, 0.0541184, 0.0, 1.0, -0.0579395], 40)